<a href="https://colab.research.google.com/github/Supriya-Konijeti/Movie-Recom-System/blob/master/4_1_MovieRecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

ratingData = pd.io.parsers.read_csv('ratings.dat', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')
movieData = pd.io.parsers.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::',encoding='latin-1')
print(ratingData)


ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)


normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
#A = ratingMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)


# A = ratingMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
# U, S, V = np.linalg.svd(A)


def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]


k = 50
movie_id = 4
top_n = 10

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

        user_id  movie_id  rating       time
0             1      1193       5  978300760
1             1       661       3  978302109
2             1       914       3  978301968
3             1      3408       4  978300275
4             1      2355       5  978824291
...         ...       ...     ...        ...
769082     4579      3334       4  969752076
769083     4579      2394       4  969751000
769084     4579      2396       5  969750724
769085     4579      2398       5  969752108
769086     4579      3341       3  970625322

[769087 rows x 4 columns]
[[  5 199  63 ... 112   4   5]
 [ 33  23 127 ...   2  23 127]
 [  0   0 176 ...   0   0 112]
 ...
 [  0 176 237 ...   0 112 154]
 [133  20  23 ... 134  20  23]
 [127   0   0 ... 127   0   0]]
[[-42.85411662 151.14588338  15.14588338 ...  64.14588338 -43.85411662
  -42.85411662]
 [-32.37759336 -42.37759336  61.62240664 ... -63.37759336 -42.37759336
   61.62240664]
 [-63.0541603  -63.0541603  112.9458397  ... -63.0541603  -63.05416